In [53]:
import pandas as pd
import numpy as np

In [63]:
aapl = pd.read_csv('WIKI_PRICES_AAPL.csv')
googl = pd.read_csv('WIKI_PRICES_GOOGL.csv')
fool = pd.read_csv('motley_with_s_scores.csv')

In [64]:
aapl.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
0,AAPL,1980-12-12,28.75,28.87,28.75,28.75,2093900.0,0.0,1.0,0.424421,0.426193,0.424421,0.424421,117258400.0
1,AAPL,1980-12-15,27.38,27.38,27.25,27.25,785200.0,0.0,1.0,0.404196,0.404196,0.402277,0.402277,43971200.0
2,AAPL,1980-12-16,25.37,25.37,25.25,25.25,472000.0,0.0,1.0,0.374524,0.374524,0.372752,0.372752,26432000.0
3,AAPL,1980-12-17,25.87,26.00,25.87,25.87,385900.0,0.0,1.0,0.381905,0.383824,0.381905,0.381905,21610400.0
4,AAPL,1980-12-18,26.63,26.75,26.63,26.63,327900.0,0.0,1.0,0.393125,0.394896,0.393125,0.393125,18362400.0


In [65]:
def format_df(stock_df, news_df, word):
    stock_df['diff'] = stock_df['close']-stock_df['open']
    news_df['Count'] = news_df['article'].apply(lambda x: x.count(word))
    news_df = news_df[news_df['Count']>5]
    news_df['date'] = pd.to_datetime(news_df['date'])
    news_df['date'] = news_df['date'].dt.strftime('%Y-%m-%d')
    news_df = news_df.groupby(['date'], as_index = False).sum()
    news_df = news_df[['date', 'sentiment', 'Count']]
    merged_df = pd.merge(news_df, stock_df)
    merged_df['bin_sentiment'] = np.where(merged_df['sentiment']>0, '1', '-1')
    merged_df['bin_diff'] = np.where(merged_df['diff']>0, '1', '-1')
    return merged_df    

In [66]:
merged_df = format_df(aapl, fool, 'Apple')

/Users/pampatterson/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/pampatterson/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [67]:
merged_df.head()

,date,sentiment,Count,ticker,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume,diff,bin_sentiment,bin_diff
0,2017-05-01,-2,26,AAPL,145.10,147.20,144.96,146.60,32818760.0,0.0,1.0,144.508636,146.600078,144.369207,146.002523,32818760.0,1.50,-1,1
1,2017-05-02,22,98,AAPL,147.54,148.09,146.84,147.51,39752670.0,0.0,1.0,146.938692,147.486450,146.241545,146.908814,39752670.0,-0.03,1,-1
2,2017-05-03,54,188,AAPL,145.59,147.49,144.27,147.06,45142806.0,0.0,1.0,144.996639,146.888896,143.682019,146.460648,45142806.0,1.47,1,1
3,2017-05-04,14,100,AAPL,146.52,147.14,145.81,146.53,23275690.0,0.0,1.0,145.922849,146.540322,145.215743,145.932808,23275690.0,0.01,1,1
4,2017-05-05,8,48,AAPL,146.76,148.98,146.76,148.96,26787359.0,0.0,1.0,146.161871,148.372823,146.161871,148.352905,26787359.0,2.20,1,1


In [69]:
merged_df.to_csv('merged_df.csv', encoding='utf-8')